In [5]:
import os
os.chdir("/home/william-ackerman/Desktop/Nanocrystal_Models/NP_XYZ_Models")

In [6]:
import numpy as np

def xyz_to_adda_dipoles(xyz_file, grid_spacing=0.8, output_file='ag_nanoparticle.dat'):
    """
    Convert XYZ coordinates to ADDA dipole format
    
    xyz_file: your nanoparticle XYZ file
    grid_spacing: spacing between dipoles (nm)
    output_file: ADDA format file
    """
    
    # Read XYZ file (skip header lines)
    data = []
    with open(xyz_file, 'r') as f:
        lines = f.readlines()
        n_atoms = int(lines[0])
        for i in range(2, 2 + n_atoms):  # Skip first 2 header lines
            parts = lines[i].split()
            element = parts[0]
            x, y, z = float(parts[1]), float(parts[2]), float(parts[3])
            data.append([element, x, y, z])
    
    data = np.array(data)
    positions = data[:, 1:4].astype(float)  # x,y,z coordinates
    elements = data[:, 0]  # element types
    
    # Center the structure
    center = np.mean(positions, axis=0)
    positions_centered = positions - center
    
    # Convert to grid coordinates
    grid_coords = np.round(positions_centered / grid_spacing).astype(int)
    
    # Shift to make all coordinates positive
    min_coords = np.min(grid_coords, axis=0)
    grid_coords_positive = grid_coords - min_coords
    
    # Remove duplicates (multiple atoms might map to same grid point)
    unique_coords = np.unique(grid_coords_positive, axis=0)
    
    # Write ADDA format (header + x y z only)
    with open(output_file, 'w') as f:
        f.write("#generated for ADDA\n")
        f.write(f"#shape: {elements[0]} nanoparticle from {xyz_file}\n")
        f.write(f"#total dipoles: {len(unique_coords)}\n")
        f.write(f"#grid spacing: {grid_spacing} nm\n")
        f.write("#lattice spacings: 1 1 1\n")
        
        # Write coordinates (just x y z, no material index)
        for coord in unique_coords:
            f.write(f"{coord[0]} {coord[1]} {coord[2]}\n")
    
    print(f"Converted {len(positions)} atoms to {len(unique_coords)} dipoles")
    print(f"Grid dimensions: {np.max(unique_coords, axis=0) + 1}")
    
    return len(unique_coords)

In [ ]:
import numpy as np
import subprocess
import pandas as pd
from scipy.interpolate import interp1d

"""
#NOTE: This is is in eps format, not RI
"""

# Johnson & Christy 1972 silver data (wavelength in nm, real and imaginary parts of dielectric constant)
silver_data = np.array([
    [300,  0.839, 2.653],
    [310,  0.901, 1.497],
    [320,  0.502, 0.635],
    [330, -0.495, 0.418],
    [340, -1.094, 0.313],
    [350, -1.725, 0.302],
    [360, -2.304, 0.265],
    [370, -2.839, 0.226],
    [380, -3.364, 0.194],
    [390, -3.896, 0.197],
    [400, -4.442, 0.211],
    [410, -4.999, 0.224],
    [420, -5.508, 0.217],
    [430, -5.991, 0.199],
    [440, -6.497, 0.204],
    [450, -7.009, 0.212],
    [460, -7.573, 0.245],
    [470, -8.164, 0.283],
    [480, -8.693, 0.295],
    [490, -9.236, 0.304],
    [500, -9.797, 0.313],
    [510, -10.378, 0.322],
    [520, -10.974, 0.332],
    [530, -11.605, 0.363],
    [540, -12.247, 0.398],
    [550, -12.908, 0.429],
    [560, -13.508, 0.417],
    [570, -14.118, 0.403],
    [580, -14.740, 0.389],
    [590, -15.380, 0.410],
    [600, -16.053, 0.442],
    [650, -19.685, 0.461],
    [700, -22.348, 0.395],
    [750, -26.000, 0.324],
    [800, -30.699, 0.410],
])

# Palladium optical constants (compiled from multiple sources)
platinum_data = np.array([
    [300.0, -3.778937, 5.469852],
    [310.0, -4.032655, 5.667792],
    [320.0, -4.392300, 5.856400],
    [330.0, -4.739509, 5.976291],
    [340.0, -5.054480, 6.216192],
    [350.0, -5.377164, 6.435912],
    [360.0, -5.704952, 6.636931],
    [370.0, -6.038427, 6.846485],
    [380.0, -6.386037, 7.105470],
    [390.0, -6.685633, 7.403231],
    [400.0, -6.990470, 7.699923],
    [410.0, -7.313657, 7.976048],
    [420.0, -7.631444, 8.269190],
    [430.0, -7.950444, 8.572929],
    [440.0, -8.275092, 8.866544],
    [450.0, -8.606052, 9.163264],
    [460.0, -8.960386, 9.507502],
    [470.0, -9.323766, 9.862903],
    [480.0, -9.665997, 10.205261],
    [490.0, -10.011185, 10.551245],
    [500.0, -10.355910, 10.886694],
    [510.0, -10.696515, 11.202566],
    [520.0, -11.042591, 11.522918],
    [530.0, -11.387506, 11.884601],
    [540.0, -11.736964, 12.256027],
    [550.0, -12.093947, 12.622414],
    [560.0, -12.476490, 12.895997],
    [570.0, -12.864686, 13.172233],
    [580.0, -13.258549, 13.451110],
    [590.0, -13.635039, 13.783151],
    [600.0, -14.010913, 14.131994],
    [610.0, -14.391888, 14.485184],
    [620.0, -14.775614, 14.817675],
    [630.0, -15.158719, 15.094820],
    [640.0, -15.546542, 15.374346],
    [650.0, -15.939081, 15.656252],
    [660.0, -16.336807, 15.942197],
    [670.0, -16.743547, 16.245705],
    [680.0, -17.155160, 16.551929],
    [690.0, -17.571642, 16.860880],
    [700.0, -17.992999, 17.172569],
    [710.0, -18.382462, 17.498170],
    [720.0, -18.751340, 17.834333],
    [730.0, -19.123875, 18.173700],
    [740.0, -19.500071, 18.516263],
    [750.0, -19.879933, 18.862011],
    [760.0, -20.264054, 19.209453],
    [770.0, -20.652750, 19.557768],
    [780.0, -21.045143, 19.909212],
    [790.0, -21.441223, 20.263779],
    [800.0, -21.840987, 20.621482]
])

# Gold data (Johnson & Christy 1972)
gold_data = np.array([
    [300.0, -1.236385, 5.759822],
    [310.0, -1.241035, 5.791356],
    [320.0, -1.230804, 5.845840],
    [330.0, -1.335010, 5.618790],
    [340.0, -1.322499, 5.547848],
    [350.0, -1.260510, 5.576285],
    [360.0, -1.304135, 5.603087],
    [370.0, -1.429679, 5.614411],
    [380.0, -1.575579, 5.639521],
    [390.0, -1.628565, 5.694674],
    [400.0, -1.659306, 5.734863],
    [410.0, -1.692281, 5.721413],
    [420.0, -1.698291, 5.690805],
    [430.0, -1.692758, 5.652979],
    [440.0, -1.723187, 5.483070],
    [450.0, -1.755834, 5.300742],
    [460.0, -1.733830, 5.083171],
    [470.0, -1.705548, 4.865861],
    [480.0, -1.926650, 4.470963],
    [490.0, -2.153398, 4.058682],
    [500.0, -2.560591, 3.643486],
    [510.0, -3.240608, 3.173938],
    [520.0, -3.883858, 2.637727],
    [530.0, -4.532941, 2.460645],
    [540.0, -5.210075, 2.293091],
    [550.0, -5.906615, 2.101000],
    [560.0, -6.566337, 1.986432],
    [570.0, -7.253030, 1.850888],
    [580.0, -7.966700, 1.694364],
    [590.0, -8.666981, 1.606637],
    [600.0, -9.383507, 1.529646],
    [610.0, -10.126300, 1.441970],
    [620.0, -10.858827, 1.354443],
    [630.0, -11.527659, 1.284276],
    [640.0, -12.215506, 1.207673],
    [650.0, -12.922354, 1.124635],
    [660.0, -13.648209, 1.035160],
    [670.0, -14.339621, 1.043770],
    [680.0, -15.048044, 1.051549],
    [690.0, -15.773492, 1.058482],
    [700.0, -16.515959, 1.064576],
    [710.0, -17.235640, 1.089537],
    [720.0, -17.943587, 1.127981],
    [730.0, -18.665781, 1.167075],
    [740.0, -19.402222, 1.206818],
    [750.0, -20.152911, 1.247211],
    [760.0, -20.913351, 1.292346],
    [770.0, -21.681012, 1.344524],
    [780.0, -22.462509, 1.397727],
    [790.0, -23.257841, 1.451954],
    [800.0, -24.067001, 1.507195]
])

# Dictionary of all metal data
METAL_DATA = {
    'Ag': silver_data,
    'Au': gold_data,
    'Pd': palladium_data
}

def get_metal_refractive_index(metal, wavelength):
    """
    Get refractive index for different metals
    
    Parameters:
    metal: str ('Ag', 'Au', 'Pd')
    wavelength: float (nm)
    
    Returns:
    n_real, n_imag: float, float (real and imaginary parts of refractive index)
    """
    
    if metal not in METAL_DATA:
        available_metals = list(METAL_DATA.keys())
        raise ValueError(f"Metal '{metal}' not available. Available metals: {available_metals}")
    
    # Get the data for this metal
    metal_data = METAL_DATA[metal]
    
    # Extract wavelengths and dielectric constants
    wavelengths = metal_data[:, 0]
    eps_real = metal_data[:, 1]
    eps_imag = metal_data[:, 2]
    
    # Interpolate dielectric data
    interp_real = interp1d(wavelengths, eps_real, kind='cubic', bounds_error=False, fill_value='extrapolate')
    interp_imag = interp1d(wavelengths, eps_imag, kind='cubic', bounds_error=False, fill_value='extrapolate')
    
    eps_r = interp_real(wavelength)
    eps_i = interp_imag(wavelength)
    
    # Convert ε to n: n = sqrt(ε)
    eps_complex = eps_r + 1j * eps_i
    n_complex = np.sqrt(eps_complex)
    
    return n_complex.real, n_complex.imag


def get_silver_refractive_index(wavelength):
    """Convert dielectric constant to refractive index"""
    # Interpolate dielectric data
    wavelengths = silver_data[:, 0]
    eps_real = silver_data[:, 1]
    eps_imag = silver_data[:, 2]
    
    interp_real = interp1d(wavelengths, eps_real, kind='cubic', bounds_error=False, fill_value='extrapolate')
    interp_imag = interp1d(wavelengths, eps_imag, kind='cubic', bounds_error=False, fill_value='extrapolate')
    
    eps_r = interp_real(wavelength)
    eps_i = interp_imag(wavelength)
    
    # Convert ε to n: n = sqrt(ε)
    eps_complex = eps_r + 1j * eps_i
    n_complex = np.sqrt(eps_complex)
    
    return n_complex.real, n_complex.imag

## Batch Convert

Convert all *_moderate_*.xzy files to DDA .dat files in selected directory

In [8]:
import os
import glob
import numpy as np

def get_grid_spacing(filename):
    """
    Determine grid spacing based on nanoparticle size in filename
    """
    if '10nm' in filename:
        return 0.4
    elif '25nm' in filename:
        return 0.4  # Adjust this value as appropriate
    elif '50nm' in filename:
        return 0.4  # Adjust this value as appropriate
    else:
        return 0.4  # Default value

def batch_convert_xyz_files(folder_path):
    """
    Convert all xyz files with '_moderate_' in their names to ADDA dipole format
    """
    # Change to the specified directory
    original_dir = os.getcwd()
    os.chdir(folder_path)
    
    try:
        # Find all xyz files with '_moderate_' in the name
        pattern = '*_moderate_*.xyz'
        xyz_files = glob.glob(pattern)
        
        if not xyz_files:
            print(f"No files matching pattern '{pattern}' found in {folder_path}")
            return
        
        print(f"Found {len(xyz_files)} files to convert:")
        for file in xyz_files:
            print(f"  - {file}")
        
        # Process each file
        for xyz_file in xyz_files:
            print(f"\nProcessing: {xyz_file}")
            
            # Generate output filename by replacing .xyz with .dat
            output_file = xyz_file.replace('.xyz', '.dat')
            
            # Determine grid spacing based on filename
            grid_spacing = get_grid_spacing(xyz_file)
            
            print(f"  Grid spacing: {grid_spacing} nm")
            print(f"  Output file: {output_file}")
            
            try:
                # Convert the file
                n_dipoles = xyz_to_adda_dipoles(xyz_file, 
                                               grid_spacing=grid_spacing, 
                                               output_file=output_file)
                print(f"  Successfully converted to {n_dipoles} dipoles")
                
            except Exception as e:
                print(f"  Error converting {xyz_file}: {str(e)}")
        
        print(f"\nBatch conversion complete!")
        
    finally:
        # Return to original directory
        os.chdir(original_dir)

In [9]:
# Run the batch conversion
folder_path = '/home/william-ackerman/Desktop/shortcut-to-nas/NLP_Nanomaterials/Nanocrystal_Models/NP_XYZ_Models'
batch_convert_xyz_files(folder_path)

Found 136 files to convert:
  - ag_fcc_110_stabilized_moderate_25nm.xyz
  - pd_fcc_100_110_stabilized_moderate_50nm.xyz
  - ag_fcc_all_stabilized_moderate_50nm.xyz
  - pt_fcc_100_110_stabilized_moderate_25nm.xyz
  - pd_fcc_111_stabilized_moderate_10nm.xyz
  - ag_fcc_110_111_stabilized_moderate_50nm.xyz
  - ag_fcc_100_111_stabilized_moderate_25nm.xyz
  - cu_fcc_all_stabilized_moderate_25nm.xyz
  - au_fcc_110_111_stabilized_moderate_50nm.xyz
  - ag_fcc_all_stabilized_moderate_25nm.xyz
  - pd_fcc_bare_metal_moderate_25nm.xyz
  - pd_fcc_110_stabilized_moderate_7nm.xyz
  - pd_fcc_all_stabilized_moderate_50nm.xyz
  - cu_fcc_100_110_stabilized_moderate_50nm.xyz
  - pt_fcc_110_111_stabilized_moderate_50nm.xyz
  - cu_fcc_110_stabilized_moderate_25nm.xyz
  - pd_fcc_100_110_stabilized_moderate_15nm.xyz
  - ag_fcc_111_stabilized_moderate_10nm.xyz
  - cu_fcc_bare_metal_moderate_10nm.xyz
  - au_fcc_bare_metal_moderate_10nm.xyz
  - au_fcc_100_111_stabilized_moderate_10nm.xyz
  - au_fcc_all_stabilized

In [10]:
filename = 'ag_fcc_111_stabilized_moderate_10nm.xyz'
n_dipoles = xyz_to_adda_dipoles(filename, grid_spacing=0.4, output_file='ag_np_111_10.dat')

Converted 33781 atoms to 33781 dipoles
Grid dimensions: [369 369 369]


In [10]:
filename = 'ag_fcc_100_stabilized_moderate_10nm.xyz'
n_dipoles = xyz_to_adda_dipoles(filename, grid_spacing=0.4, output_file='ag_fcc_100_stabilized_moderate_10nm.dat')

Converted 34461 atoms to 34461 dipoles
Grid dimensions: [205 205 205]


In [13]:
filename = 'ag_fcc_100_stabilized_moderate_25nm.xyz'
n_dipoles = xyz_to_adda_dipoles(filename, grid_spacing=0.4, output_file='ag_fcc_100_stabilized_moderate_25nm.dat')

Converted 485149 atoms to 485149 dipoles
Grid dimensions: [503 503 503]


In [12]:
filename = 'ag_fcc_100_stabilized_moderate_10nm.xyz'
n_dipoles = xyz_to_adda_dipoles(filename, grid_spacing=1, output_file='ag_fcc_100_stabilized_moderate_10nm.dat')

Converted 34461 atoms to 34461 dipoles
Grid dimensions: [83 83 83]


In [17]:
import subprocess
import psutil
import numpy as np

def check_gpu_available():
    """Check if NVIDIA GPU is available"""
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        return result.returncode == 0
    except FileNotFoundError:
        return False
    
def get_optimal_adda_path(n_dipoles, adda_base_path="/home/william-ackerman/adda/src"):
    """
    Choose optimal ADDA version based on system resources and problem size
    """
    
    # Check avail mem
    available_memory_gb = psutil.virtual_memory().available / (1024**3)

    # Is GPU avail?
    gpu_available = check_gpu_available()

    # Rough estimateion
    estimated_memory_gb = (n_dipoles / 1000) ** 1.3 / 100  # Empirical scaling
    
    print(f"Problem size: {n_dipoles} dipoles")
    print(f"Estimated memory needed: {estimated_memory_gb:.2f} GB")
    print(f"Available system memory: {available_memory_gb:.2f} GB")


    if gpu_available and n_dipoles < 100000 and estimated_memory_gb < 3:
        # Use GPU for medium-sized problems
        ADDA_PATH = f"{adda_base_path}/ocl/adda_ocl"
        print("Selected: GPU (OpenCL) version")
            
    elif n_dipoles < 600000 and available_memory_gb < 115:
        # Use MPI for large problems with sufficient RAM
        ADDA_PATH = f"{adda_base_path}/mpi/adda_mpi"
        print("Selected: MPI (multi-CPU) version")
            
    else:
        # Fall back to sequential for small problems or limited resources
        ADDA_PATH = f"{adda_base_path}/seq/adda"
        print("Selected: Sequential (single-CPU) version")    

    print(ADDA_PATH)

    return ADDA_PATH

def parse_adda_output(stdout_text):
    """Extract cross-sections from ADDA stdout"""
    lines = stdout_text.split('\n')
    
    cext = qext = cabs = qabs = np.nan
    
    for line in lines:
        if 'Cext' in line and '=' in line:
            cext = float(line.split('=')[1].strip())
        elif 'Qext' in line and '=' in line:
            qext = float(line.split('=')[1].strip())
        elif 'Cabs' in line and '=' in line:
            cabs = float(line.split('=')[1].strip())
        elif 'Qabs' in line and '=' in line:
            qabs = float(line.split('=')[1].strip())
    
    return cext, qext, cabs, qabs

def run_adda_with_medium(shape_file, wavelengths, ADDA_PATH, metal='Ag', medium_n=1.33, base_dir="adda_results_medium"):
    """
    Run ADDA with medium effects (solvent) for different metals
    
    Parameters:
    shape_file: str - ADDA dipole file
    wavelengths: array - wavelengths to scan (nm)
    ADDA_PATH: str - path to ADDA executable
    metal: str - metal type ('Ag', 'Au', 'Pd')
    medium_n: float - medium refractive index (1.0=vacuum, 1.33=water)
    base_dir: str - output directory
    """
    import os
    import subprocess
    import numpy as np
    import pandas as pd
    
    # Create results directory
    os.makedirs(base_dir, exist_ok=True)
    
    results = []  # Initialize the results list!
    
    for wl in wavelengths:
        print(f"Running wavelength {wl} nm for {metal} in medium (n={medium_n})...")
        
        # Get metal refractive index at this wavelength
        n_metal_real, n_metal_imag = get_metal_refractive_index(metal, wl)
        
        # Relative refractive index: n_particle/n_medium
        n_rel_real = n_metal_real / medium_n
        n_rel_imag = n_metal_imag / medium_n
        
        # Create unique run directory
        run_dir = f"{base_dir}/{metal}_wl_{wl}nm"
        
        cmd = [ADDA_PATH, "-shape", "read", shape_file,
               "-lambda", str(wl),
               "-m", f"{n_rel_real:.6f}", f"{n_rel_imag:.6f}",
               "-orient", "avg",
               "-dir", run_dir]
        
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, cwd=".")
            
            if result.returncode == 0:
                # Parse the output
                cext, qext, cabs, qabs = parse_adda_output(result.stdout)
                csca = cext - cabs
                qsca = qext - qabs
                
                # Store results with metal info
                results.append([wl, metal, n_rel_real, n_rel_imag, cext, qext, cabs, qabs, csca, qsca])
                print(f"  ✓ λ={wl}nm: Qext={qext:.3f}")
            else:
                print(f"  ✗ Error at λ={wl}nm: {result.stderr}")
                results.append([wl, metal, n_rel_real, n_rel_imag, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
                
        except Exception as e:
            print(f"  ✗ Exception at λ={wl}nm: {e}")
            n_rel_real, n_rel_imag = get_metal_refractive_index(metal, wl)  # Get values for error case
            n_rel_real /= medium_n
            n_rel_imag /= medium_n
            results.append([wl, metal, n_rel_real, n_rel_imag, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
    
    # Convert to DataFrame for easy handling
    columns = ['wavelength', 'metal', 'n_real', 'n_imag', 'Cext', 'Qext', 'Cabs', 'Qabs', 'Csca', 'Qsca']
    df = pd.DataFrame(results, columns=columns)
    
    return df

def get_n_dipoles_from_file(shape_file):
    """
    Count number of dipoles in ADDA shape file
    """
    if not os.path.exists(shape_file):
        print(f"Warning: {shape_file} not found")
        return 0
    
    n_dipoles = 0
    with open(shape_file, 'r') as f:
        for line in f:
            # Skip comment lines and empty lines
            line = line.strip()
            if line and not line.startswith('#'):
                n_dipoles += 1
    
    return n_dipoles

## Orientation Averaging

In [64]:
run_dir = os.getcwd()

avg_params_content = """alpha:
min=0
max=360
Jmin=3
Jmax=5
eps=1e-5
equiv=false
periodic=true
beta:
min=0
max=180
Jmin=3
Jmax=4
eps=1e-5
equiv=false
periodic=false
gamma:
min=0
max=0
Jmin=1
Jmax=1
eps=1e-5
equiv=false
periodic=true
"""

avg_params_path = os.path.join(run_dir, "avg_params.dat")
with open(avg_params_path, 'w') as f:
    f.write(avg_params_content)

In [26]:
wavelengths = np.arange(300, 501, 2.5)  # 300-500 nm in 2.5 nm steps

num_dipoles = get_n_dipoles_from_file("ag_np_100_10.dat")

ADDA_PATH = get_optimal_adda_path(num_dipoles)

results_df_10 = run_adda_with_medium("ag_np_100_10.dat", wavelengths, ADDA_PATH, medium_n=1.33)

Problem size: 34461 dipoles
Estimated memory needed: 1.00 GB
Available system memory: 114.91 GB
Selected: GPU (OpenCL) version
/home/william-ackerman/adda/src/ocl/adda_ocl
Running wavelength 300.0 nm for Ag in medium (n=1.33)...
  ✓ λ=300.0nm: Qext=19.264
Running wavelength 302.5 nm for Ag in medium (n=1.33)...
  ✓ λ=302.5nm: Qext=18.752
Running wavelength 305.0 nm for Ag in medium (n=1.33)...
  ✓ λ=305.0nm: Qext=18.006
Running wavelength 307.5 nm for Ag in medium (n=1.33)...
  ✓ λ=307.5nm: Qext=17.023
Running wavelength 310.0 nm for Ag in medium (n=1.33)...
  ✓ λ=310.0nm: Qext=15.773
Running wavelength 312.5 nm for Ag in medium (n=1.33)...
  ✓ λ=312.5nm: Qext=14.606
Running wavelength 315.0 nm for Ag in medium (n=1.33)...
  ✓ λ=315.0nm: Qext=13.931
Running wavelength 317.5 nm for Ag in medium (n=1.33)...
  ✓ λ=317.5nm: Qext=13.179
Running wavelength 320.0 nm for Ag in medium (n=1.33)...
  ✓ λ=320.0nm: Qext=13.693
Running wavelength 322.5 nm for Ag in medium (n=1.33)...
  ✓ λ=322.5nm: 

In [27]:
import pickle

with open('UV-Vis_Sim_Ag_10nm_100_rf_1.3.AVG_ORIENT.pkl', 'wb') as f:
    pickle.dump(results_df_10, f)

In [34]:
max(results_df_10.Qext)

112.0663146

##NOTES   
Okay, so we can get identical tracings for 10nm locally and on Lakeshore for en face orientation
We are testing orientation averaging on Zen 5
DO WE HAVE ALL OF THE RI DATA FROM SOURCES CORRECTED FOR THESE ANALYSES? n,k data from https://refractiveindex.info/

https://refractiveindex.info/database/data/main/Cu/nk/Johnson.yml
https://refractiveindex.info/database/data/main/Ag/nk/Johnson.yml
https://refractiveindex.info/database/data/main/Au/nk/Johnson.yml
https://refractiveindex.info/database/data/main/Pd/nk/Johnson.yml
https://refractiveindex.info/database/data/main/Pt/nk/Werner.yml



In [ ]:
# Quick plot
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(results_df_10['wavelength'], results_df_10['Qext'], '-', label='Qext')
plt.plot(results_df_10['wavelength'], results_df_10['Qabs'], '-', label='Qabs') 
plt.plot(results_df_10['wavelength'], results_df_10['Qsca'], '-', label='Qsca')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Efficiency')
plt.legend()
plt.title('Silver Nanocube 10 nm in Medium 0.4 Grid Spacing Optical Response - WITH ORIENTATION AVERAGING')
plt.grid(True)
plt.show()

In [15]:
import pickle

# Define wavelength range
wavelengths = np.arange(300, 501, 2.5)  # 300-500 nm in 2.5 nm steps

num_dipoles = get_n_dipoles_from_file("ag_np_100_10.dat")

ADDA_PATH = get_optimal_adda_path(num_dipoles)

results_df_10 = run_adda_with_medium("ag_np_100_10.dat", wavelengths, ADDA_PATH, medium_n=1.33)

with open('UV-Vis_Sim_Ag_10nm_100_rf_1.3.INTERVAL_WIDER.pkl', 'wb') as f:
    pickle.dump(results_df_10, f)



Problem size: 34461 dipoles
Estimated memory needed: 1.00 GB
Available system memory: 113.05 GB
Selected: GPU (OpenCL) version
/home/william-ackerman/adda/src/ocl/adda_ocl
Running wavelength 300.0 nm for Ag in medium (n=1.33)...
  ✓ λ=300.0nm: Qext=19.375
Running wavelength 302.5 nm for Ag in medium (n=1.33)...
  ✓ λ=302.5nm: Qext=18.847
Running wavelength 305.0 nm for Ag in medium (n=1.33)...
  ✓ λ=305.0nm: Qext=18.071
Running wavelength 307.5 nm for Ag in medium (n=1.33)...
  ✓ λ=307.5nm: Qext=16.986
Running wavelength 310.0 nm for Ag in medium (n=1.33)...
  ✓ λ=310.0nm: Qext=16.138
Running wavelength 312.5 nm for Ag in medium (n=1.33)...
  ✓ λ=312.5nm: Qext=14.804
Running wavelength 315.0 nm for Ag in medium (n=1.33)...
  ✓ λ=315.0nm: Qext=13.794
Running wavelength 317.5 nm for Ag in medium (n=1.33)...
  ✓ λ=317.5nm: Qext=12.797
Running wavelength 320.0 nm for Ag in medium (n=1.33)...
  ✓ λ=320.0nm: Qext=15.053
Running wavelength 322.5 nm for Ag in medium (n=1.33)...
  ✓ λ=322.5nm: 

# UV-VIS SIMULATIONS 
More detail with 1 nm incremenets vs 5 nm increments

In [ ]:
# Quick plot
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(results_df_10['wavelength'], results_df_10['Qext'], '-', label='Qext')
plt.plot(results_df_10['wavelength'], results_df_10['Qabs'], '-', label='Qabs') 
plt.plot(results_df_10['wavelength'], results_df_10['Qsca'], '-', label='Qsca')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Efficiency')
plt.legend()
plt.title('Silver Nanocube 10 nm in Medium 0.4 Grid Spacing Optical Response')
plt.grid(True)
plt.show()

In [ ]:
# Quick plot
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(results_df_10['wavelength'], results_df_10['Qext'], '-', label='Qext')
plt.plot(results_df_10['wavelength'], results_df_10['Qabs'], '-', label='Qabs') 
plt.plot(results_df_10['wavelength'], results_df_10['Qsca'], '-', label='Qsca')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Efficiency')
plt.legend()
plt.title('Silver Nanocube 10 nm in Medium 0.4 Grid Spacing Optical Response')
plt.grid(True)
plt.show()

In [ ]:
# Quick plot
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(results_df_10['wavelength'], results_df_10['Qext'], '-', label='Qext')
plt.plot(results_df_10['wavelength'], results_df_10['Qabs'], '-', label='Qabs') 
plt.plot(results_df_10['wavelength'], results_df_10['Qsca'], '-', label='Qsca')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Efficiency')
plt.legend()
plt.title('Silver Nanocube 10 nm in Medium 0.4 Grid Spacing Optical Response')
plt.grid(True)
plt.show()

## HOTSPOT DETECTION 
Can we identify "hotspots" in the smaller structure at 400 nm and selectively downsample to see if this approximation captures essentials without running into scaling issues and memory overruns?

The code is designed to identify regions of enhanced electromagnetic fields ("hot spots") in nanocrystal structures - areas where the local electric field intensity is significantly higher than average. These hot spots are crucial for applications like Surface-Enhanced Raman Spectroscopy (SERS) and plasmonic sensing.
Key Components
1. ADDA Execution with Field Extraction (run_adda_with_hotspots)

Runs ADDA simulations with specific flags to save geometric and field data
Uses -save_geom to output dipole positions
Uses -int fcd (Filtered Coupled Dipoles) to calculate internal fields
Handles metal optical properties and medium refractive index

2. Data Extraction (extract_adda_hotspots)
This is the core function that tries multiple strategies to get field enhancement data:
Primary approach: Read internal field data from ADDA output files

Looks for files containing field information
Parses complex electric field components (Ex, Ey, Ez)
Calculates field enhancement as |E|² = |Ex|² + |Ey|² + |Ez|²

Fallback approach: If no field data is available, uses geometric approximation

Estimates enhancement based on proximity to edges, corners, and faces
Assumes higher fields near sharp features (which is physically reasonable)

3. Shape File Reading (read_adda_shape_file)

Handles ADDA shape files with comments and different formats
Extracts 3D coordinates of each dipole in the discretized structure

4. Geometric Enhancement Estimation (estimate_geometric_enhancement)
When actual field data isn't available, this provides a physics-based approximation:

Calculates distances to nearest faces, edges, and corners
Assigns higher enhancement values to positions near sharp geometric features
Uses the principle that field enhancement increases near high-curvature regions

5. 3D Visualization (visualize_hotspots_3d)

Creates 3D scatter plots colored by field enhancement
Highlights hotspots above a threshold with special markers
Shows optical cross-sections (Qext, Qabs, Qsca) in the title

The Physics Behind It
Hot spots occur because:

Lightning rod effect: Sharp features (corners, edges) concentrate electric fields
Plasmonic resonances: Metal nanoparticles support surface plasmons that can create intense local fields
Gap modes: Narrow gaps between metal features can trap and amplify electromagnetic energy

Data Flow

Run ADDA simulation with field extraction enabled
Try to read actual electromagnetic field data from output files
If that fails, fall back to geometric approximation
Calculate field enhancement (|E|²) for each dipole
Visualize results, highlighting regions with high enhancement

Practical Applications
This analysis helps identify:

Optimal locations for placing molecules in SERS experiments
Design parameters for plasmonic sensors
Understanding of how nanoparticle shape affects field enhancement
Validation of theoretical predictions about hot spot locations

The code is quite robust, with multiple fallback strategies to handle different ADDA versions and output formats, which is important since ADDA's output format can vary between versions and compilation options.

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import subprocess
import os
import re

def get_material_constants_for_adda(metal, wavelength, medium_n=1.33):
    """
    Get material constants in the format needed for ADDA
    Uses your existing material database
    """
    # Get the complex refractive index
    n_real, n_imag = get_metal_refractive_index(metal, wavelength)
    
    print(f"{metal} at {wavelength} nm: n = {n_real:.3f} + {n_imag:.3f}i")
    
    # ADDA needs relative refractive index (particle/medium)
    n_rel_real = n_real / medium_n
    n_rel_imag = n_imag / medium_n
    
    print(f"Relative to medium (n={medium_n}): m = {n_rel_real:.3f} + {n_rel_imag:.3f}i")
    
    return n_rel_real, n_rel_imag

# Updated main analysis function using your material database
def analyze_nanocube_enhancement_complete(shape_file, wavelength=408, metal='Ag', medium_n=1.33,
                                        # Visualization parameters
                                        size_scale=2, size_multiplier=3, opacity=0.8,
                                        colorscale='hot', show_full_3d=True, show_surface=True, 
                                        show_cross_sections=True, show_scattering=True):
    """
    Complete analysis with full visualization control
    
    Visualization Parameters:
    ------------------------
    size_scale : float
        Base marker size
    size_multiplier : float  
        Enhancement-based size multiplier
    opacity : float
        Marker opacity (0-1)
    colorscale : str
        Color scheme ('hot', 'plasma', 'viridis', etc.)
    show_full_3d : bool
        Show full 3D visualization
    show_surface : bool
        Show surface-only visualization  
    show_cross_sections : bool
        Show cross-sectional views
    show_scattering : bool
        Show scattering contribution plots
    """
    print(f"=== Analyzing {shape_file} ({metal}) at {wavelength} nm ===")
    
    # Get material constants
    n_rel_real, n_rel_imag = get_material_constants_for_adda(metal, wavelength, medium_n)
    
    # Run ADDA
    adda_result = analyze_single_wavelength_with_fields(
        shape_file, wavelength, n_rel_real=n_rel_real, n_rel_imag=n_rel_imag
    )
    
    if adda_result is None:
        return None
    
    # Parse cross-sections
    qext, qabs, qsca = parse_cross_sections_from_file()
    if qext is not None:
        adda_result.update({'qext': qext, 'qabs': qabs, 'qsca': qsca})
        print(f"Cross-sections: Qext={qext:.1f}, Qabs={qabs:.1f}, Qsca={qsca:.1f}")
    
    # Read field data
    data = read_dipole_fields_and_polarizations()
    if data is None:
        return None
    
    results = {
        'adda_result': adda_result,
        'data': data,
        'metal': metal,
        'wavelength': wavelength
    }
    
    # Create visualizations based on flags
    if 'field_enhancement' in data:
        print("Creating field enhancement visualizations...")
        
        if show_full_3d:
            fig_full = visualize_enhancement_3d(
                data['positions'], 
                data['field_enhancement'],
                title=f"{metal} Nanocube Full 3D Enhancement at {wavelength} nm (Qext={qext:.1f})",
                size_scale=size_scale,
                size_multiplier=size_multiplier,
                opacity=opacity,
                colorscale=colorscale
            )
            results['fig_full'] = fig_full
            fig_full.show()
        
        if show_surface:
            fig_surface = create_surface_view(
                data['positions'],
                data['field_enhancement'],
                title=f"{metal} Nanocube Surface Enhancement at {wavelength} nm (Qext={qext:.1f})",
                size_scale=size_scale,
                size_multiplier=size_multiplier,
                opacity=opacity,
                colorscale=colorscale
            )
            results['fig_surface'] = fig_surface
            fig_surface.show()
        
        if show_cross_sections:
            fig_cross = create_cross_section_plots(
                data['positions'], 
                data['field_enhancement'],
                title=f"{metal} Nanocube Cross-sections at {wavelength} nm (Qext={qext:.1f})",
                colorscale=colorscale
            )
            results['fig_cross_sections'] = fig_cross
            fig_cross.show()
    
    if show_scattering and 'scattering_contrib' in data:
        print("Creating scattering contribution visualizations...")
        
        fig_sca_surface = create_surface_view(
            data['positions'],
            data['scattering_contrib'],
            title=f"{metal} Nanocube Scattering Contribution at {wavelength} nm (Qsca={qsca:.1f})",
            size_scale=size_scale,
            size_multiplier=size_multiplier,
            opacity=opacity,
            colorscale=colorscale
        )
        results['fig_scattering_surface'] = fig_sca_surface
        fig_sca_surface.show()
    
    return results

# Function to compare different metals
def compare_metals_at_wavelength(shape_file, wavelength=408, metals=['Ag', 'Au', 'Pd'], medium_n=1.33):
    """
    Compare field enhancement for different metals at the same wavelength
    """
    results = {}
    
    for metal in metals:
        print(f"\n{'='*50}")
        print(f"Analyzing {metal}")
        print(f"{'='*50}")
        
        try:
            result = analyze_nanocube_enhancement_complete(
                shape_file, wavelength, metal, medium_n
            )
            results[metal] = result
        except Exception as e:
            print(f"Error analyzing {metal}: {e}")
            results[metal] = None
    
    return results

# Function to analyze wavelength series for a given metal
def analyze_wavelength_series(shape_file, wavelengths, metal='Ag', medium_n=1.33):
    """
    Analyze field enhancement across multiple wavelengths
    """
    results = {}
    
    for wl in wavelengths:
        print(f"\n{'='*50}")
        print(f"Analyzing {metal} at {wl} nm")
        print(f"{'='*50}")
        
        try:
            result = analyze_nanocube_enhancement_complete(
                shape_file, wl, metal, medium_n
            )
            results[wl] = result
        except Exception as e:
            print(f"Error analyzing {wl} nm: {e}")
            results[wl] = None
    
    return results

def visualize_enhancement_3d(positions, enhancement, title="Field Enhancement", 
                           size_scale=2, size_multiplier=3, opacity=0.8, 
                           colorscale='hot', show_colorbar=True, 
                           colorbar_title="Field Enhancement |E|²/|E₀|²",
                           width=800, height=600, camera_eye=None):
    """
    Create 3D visualization of field enhancement using plotly
    Now with full customization options
    
    Parameters:
    -----------
    positions : array
        Dipole positions
    enhancement : array  
        Enhancement values
    title : str
        Plot title
    size_scale : float
        Base size for markers
    size_multiplier : float
        Multiplier for enhancement-based sizing
    opacity : float
        Marker opacity (0-1)
    colorscale : str
        Plotly colorscale name
    show_colorbar : bool
        Whether to show colorbar
    colorbar_title : str
        Title for colorbar
    width, height : int
        Plot dimensions
    camera_eye : dict or None
        Camera position, e.g. dict(x=1.5, y=1.5, z=1.5)
    """
    if camera_eye is None:
        camera_eye = dict(x=1.5, y=1.5, z=1.5)
    
    # Normalize enhancement for color scale
    enhancement_norm = enhancement / np.max(enhancement) if np.max(enhancement) > 0 else enhancement
    
    # Create size array (larger points for higher enhancement)
    sizes = size_scale + size_multiplier * enhancement_norm
    
    fig = go.Figure(data=go.Scatter3d(
        x=positions[:, 0],
        y=positions[:, 1], 
        z=positions[:, 2],
        mode='markers',
        marker=dict(
            size=sizes,
            color=enhancement,
            colorscale=colorscale,
            opacity=opacity,
            colorbar=dict(
                title=colorbar_title,
                x=1.02,
                thickness=15,
                len=0.8
            ) if show_colorbar else None,
            showscale=show_colorbar
        ),
        text=[f'Enhancement: {enh:.2f}' for enh in enhancement],
        hovertemplate='<b>Position</b>: (%{x:.1f}, %{y:.1f}, %{z:.1f})<br>' +
                      '<b>Enhancement</b>: %{marker.color:.2f}<br>' +
                      '<extra></extra>'
    ))
    
    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,
            font=dict(size=16)
        ),
        scene=dict(
            xaxis_title='X (dipole units)',
            yaxis_title='Y (dipole units)', 
            zaxis_title='Z (dipole units)',
            camera=dict(eye=camera_eye),
            bgcolor='white'
        ),
        width=width,
        height=height
    )
    
    return fig

def create_surface_view(positions, enhancement, title="Surface Enhancement",
                       size_scale=3, size_multiplier=5, opacity=0.8,
                       interior_opacity=0.05, interior_size=1,
                       colorscale='hot', show_interior=True,
                       width=800, height=600, camera_eye=None):
    """
    Create a view focusing on the surface dipoles where enhancement is highest
    Now with full customization
    
    Parameters:
    -----------
    size_scale : float
        Base size for surface markers
    size_multiplier : float
        Multiplier for enhancement-based sizing of surface markers
    opacity : float
        Surface marker opacity
    interior_opacity : float
        Interior marker opacity
    interior_size : float
        Size of interior markers
    show_interior : bool
        Whether to show interior dipoles
    """
    if camera_eye is None:
        camera_eye = dict(x=1.5, y=1.5, z=1.5)
    
    # Find surface dipoles (those at the edges of the cube)
    min_coords = np.min(positions, axis=0)
    max_coords = np.max(positions, axis=0)
    
    tolerance = 2  # dipole units
    
    # Identify surface dipoles
    surface_mask = (
        (np.abs(positions[:, 0] - min_coords[0]) < tolerance) |  # Left face
        (np.abs(positions[:, 0] - max_coords[0]) < tolerance) |  # Right face
        (np.abs(positions[:, 1] - min_coords[1]) < tolerance) |  # Front face
        (np.abs(positions[:, 1] - max_coords[1]) < tolerance) |  # Back face
        (np.abs(positions[:, 2] - min_coords[2]) < tolerance) |  # Bottom face
        (np.abs(positions[:, 2] - max_coords[2]) < tolerance)    # Top face
    )
    
    surface_positions = positions[surface_mask]
    surface_enhancement = enhancement[surface_mask]
    interior_positions = positions[~surface_mask]
    
    # Create visualization
    fig = go.Figure()
    
    # Plot interior dipoles lightly (optional)
    if show_interior and len(interior_positions) > 0:
        fig.add_trace(
            go.Scatter3d(
                x=interior_positions[:, 0],
                y=interior_positions[:, 1],
                z=interior_positions[:, 2],
                mode='markers',
                marker=dict(
                    size=interior_size,
                    color='lightgray',
                    opacity=interior_opacity
                ),
                name="Interior",
                showlegend=False,
                hoverinfo='skip'
            )
        )
    
    # Plot surface dipoles with enhancement
    sizes_surface = size_scale + size_multiplier * (surface_enhancement / np.max(surface_enhancement))
    fig.add_trace(
        go.Scatter3d(
            x=surface_positions[:, 0],
            y=surface_positions[:, 1],
            z=surface_positions[:, 2],
            mode='markers',
            marker=dict(
                size=sizes_surface,
                color=surface_enhancement,
                colorscale=colorscale,
                opacity=opacity,
                colorbar=dict(
                    title="Surface Enhancement",
                    thickness=15,
                    len=0.8
                )
            ),
            name="Surface",
            text=[f'Enhancement: {enh:.3f}' for enh in surface_enhancement],
            hovertemplate='<b>Surface Position</b>: (%{x:.1f}, %{y:.1f}, %{z:.1f})<br>' +
                          '<b>Enhancement</b>: %{marker.color:.3f}<br>' +
                          '<extra></extra>'
        )
    )
    
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X (dipole units)',
            yaxis_title='Y (dipole units)',
            zaxis_title='Z (dipole units)',
            camera=dict(eye=camera_eye)
        ),
        width=width,
        height=height
    )
    
    return fig

def create_cross_section_plots(positions, enhancement, title="Field Enhancement",
                             marker_size=8, colorscale='hot', 
                             tolerance=2, height=800, width=1000):
    """
    Create cross-sectional views of the enhancement
    Now with customizable parameters
    
    Parameters:
    -----------
    marker_size : float
        Size of markers in 2D cross-sections
    tolerance : float
        Thickness of cross-sectional slices
    """
    # Find the center and dimensions of the cube
    center_x, center_y, center_z = np.mean(positions, axis=0)
    
    # Create cross-sections at the center
    # XY plane (middle Z)
    xy_mask = np.abs(positions[:, 2] - center_z) < tolerance
    xy_positions = positions[xy_mask]
    xy_enhancement = enhancement[xy_mask]
    
    # XZ plane (middle Y) 
    xz_mask = np.abs(positions[:, 1] - center_y) < tolerance
    xz_positions = positions[xz_mask]
    xz_enhancement = enhancement[xz_mask]
    
    # YZ plane (middle X)
    yz_mask = np.abs(positions[:, 0] - center_x) < tolerance
    yz_positions = positions[yz_mask]
    yz_enhancement = enhancement[yz_mask]
    
    # Create subplot with 3 cross-sections
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"type": "scatter3d", "colspan": 2}, None],
               [{"type": "scatter"}, {"type": "scatter"}]],
        subplot_titles=[
            "3D View", 
            "XY Cross-section (Z=center)",
            "XZ Cross-section (Y=center)"
        ],
        vertical_spacing=0.1,
        horizontal_spacing=0.1
    )
    
    # 3D view (reduced size)
    sizes_3d = 1 + 2 * (enhancement / np.max(enhancement))
    fig.add_trace(
        go.Scatter3d(
            x=positions[:, 0],
            y=positions[:, 1],
            z=positions[:, 2],
            mode='markers',
            marker=dict(
                size=sizes_3d,
                color=enhancement,
                colorscale=colorscale,
                opacity=0.6,
                showscale=True,
                colorbar=dict(
                    title="Enhancement",
                    x=0.45,
                    thickness=15,
                    len=0.4
                )
            ),
            name="3D"
        ),
        row=1, col=1
    )
    
    # XY cross-section
    if len(xy_positions) > 0:
        fig.add_trace(
            go.Scatter(
                x=xy_positions[:, 0],
                y=xy_positions[:, 1],
                mode='markers',
                marker=dict(
                    size=marker_size,
                    color=xy_enhancement,
                    colorscale=colorscale,
                    showscale=False
                ),
                name="XY"
            ),
            row=2, col=1
        )
    
    # XZ cross-section
    if len(xz_positions) > 0:
        fig.add_trace(
            go.Scatter(
                x=xz_positions[:, 0],
                y=xz_positions[:, 2],
                mode='markers',
                marker=dict(
                    size=marker_size,
                    color=xz_enhancement,
                    colorscale=colorscale,
                    showscale=False
                ),
                name="XZ"
            ),
            row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        title=title,
        height=height,
        width=width
    )
    
    # Update axes
    fig.update_xaxes(title_text="X (dipole units)", row=2, col=1)
    fig.update_yaxes(title_text="Y (dipole units)", row=2, col=1)
    fig.update_xaxes(title_text="X (dipole units)", row=2, col=2)
    fig.update_yaxes(title_text="Z (dipole units)", row=2, col=2)
    
    return fig

def run_adda_with_field_extraction(shape_file, wavelength, medium_n=1.33, 
                                  material="Ag", adda_path=None):
    """
    Run ADDA with field extraction to get dipole-level information
    Uses your existing material handling approach
    """
    if adda_path is None:
        adda_path = get_optimal_adda_path(get_n_dipoles_from_file(shape_file))
    
    # Get material properties - you'll need to adapt this to your existing material function
    # For now, let's use a simplified approach that works with your existing code
    
    # ADDA command with field extraction
    cmd = [
        adda_path,
        "-shape", f"read {shape_file}",
        "-lambda", str(wavelength),
        "-m", f"1.33 0",  # Medium refractive index (you may need to adjust this)
        "-save_geom",
        "-int_surf", 
        "-store_int_field",  # Store internal field
        "-store_dip_pol",    # Store dipole polarizations
        "-eps", "10",
        "-iter", "gpu"
    ]
    
    # Add material file if you have one
    # You might need to add: "-m", f"file {material_file}" based on your setup
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, 
                              timeout=300, cwd=".")
        
        if result.returncode != 0:
            print(f"ADDA error: {result.stderr}")
            return None
            
        # Parse cross-sections from output using your existing parsing
        # You'll need to adapt this to use your existing parse function
        output_lines = result.stdout.split('\n')
        qext = qabs = qsca = 0
        
        for line in output_lines:
            if 'Qext' in line:
                # Parse using your existing method
                pass
                
        return {
            'qext': qext,
            'qabs': qabs, 
            'qsca': qsca,
            'stdout': result.stdout,
            'stderr': result.stderr
        }
        
    except subprocess.TimeoutExpired:
        print("ADDA calculation timed out")
        return None

# Alternative approach: Modify your existing run_adda_with_medium function
def run_adda_with_medium_and_fields(shape_file, wavelengths, adda_path, medium_n=1.33):
    """
    Extended version of your run_adda_with_medium that also extracts field data
    """
    # First, let's see what your existing function looks like
    # and then we can add field extraction to it
    pass

# For now, let's create a simpler version that works with a single wavelength
def analyze_single_wavelength_with_fields(shape_file, wavelength=408, adda_path=None, 
                                        n_rel_real=0.5, n_rel_imag=2.5, run_dir="field_analysis"):
    """
    Working field extraction using the correct ADDA options
    """
    if adda_path is None:
        num_dipoles = get_n_dipoles_from_file(shape_file)
        adda_path = get_optimal_adda_path(num_dipoles)
    
    # Create clean directory
    if os.path.exists(run_dir):
        import shutil
        shutil.rmtree(run_dir)
    os.makedirs(run_dir, exist_ok=True)
    
    print(f"Running ADDA with field extraction for {wavelength} nm...")
    
    # Build command with working options
    cmd = [
        adda_path,
        "-shape", "read", shape_file,
        "-lambda", str(wavelength),
        "-m", f"{n_rel_real:.6f}", f"{n_rel_imag:.6f}",
        "-dir", run_dir,
        "-save_geom",           # Saves geometry as read.geom
        "-store_int_field",     # Saves IntField-X and IntField-Y  
        "-store_dip_pol",       # Saves DipPol-X and DipPol-Y
        "-eps", "10"
    ]
    
    try:
        print(f"Running command: {' '.join(cmd)}")
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        
        if result.returncode != 0:
            print(f"ADDA failed with error: {result.stderr}")
            print(f"ADDA stdout: {result.stdout}")
            return None
            
        print("ADDA completed successfully!")
        
        # Parse cross-sections from output
        qext = qabs = qsca = None
        for line in result.stdout.split('\n'):
            if 'Qext' in line and 'Qabs' in line and 'Qsca' in line:
                # Parse line like: "Qext=2.345, Qabs=1.234, Qsca=1.111"
                import re
                qext_match = re.search(r'Qext=([0-9.]+)', line)
                qabs_match = re.search(r'Qabs=([0-9.]+)', line)  
                qsca_match = re.search(r'Qsca=([0-9.]+)', line)
                
                if qext_match and qabs_match and qsca_match:
                    qext = float(qext_match.group(1))
                    qabs = float(qabs_match.group(1))
                    qsca = float(qsca_match.group(1))
                    break
        
        print(f"Cross-sections: Qext={qext}, Qabs={qabs}, Qsca={qsca}")
        
        # List created files
        print(f"\nFiles created in {run_dir}:")
        for file in sorted(os.listdir(run_dir)):
            file_path = os.path.join(run_dir, file)
            if os.path.isfile(file_path):
                size = os.path.getsize(file_path)
                print(f"  ✓ {file} ({size:,} bytes)")
        
        return {
            'result': result,
            'qext': qext,
            'qabs': qabs, 
            'qsca': qsca
        }
        
    except Exception as e:
        print(f"Error running ADDA: {e}")
        return None

def read_dipole_fields_and_polarizations(run_dir="field_analysis", polarization='Y'):
    """
    Read the internal fields and dipole polarizations from ADDA output files
    Fixed to handle the correct column indices for polarization data
    """
    # Correct file paths based on ADDA output
    geometry_file = os.path.join(run_dir, "read.geom")
    int_field_file = os.path.join(run_dir, f"IntField-{polarization}")
    pol_file = os.path.join(run_dir, f"DipPol-{polarization}")
    
    data = {}
    
    # Read geometry (skip header line)
    try:
        if os.path.exists(geometry_file):
            print(f"Reading geometry from {geometry_file}")
            geometry = np.loadtxt(geometry_file, skiprows=1)
            data['positions'] = geometry[:, :3]  # x, y, z positions
            print(f"Loaded {len(data['positions'])} dipole positions")
        else:
            print(f"Warning: {geometry_file} not found")
            return None
    except Exception as e:
        print(f"Error reading geometry: {e}")
        return None
    
    # Read internal fields
    try:
        if os.path.exists(int_field_file):
            print(f"Reading internal fields from {int_field_file}")
            
            # Skip header: x y z |E|^2 Ex.r Ex.i Ey.r Ey.i Ez.r Ez.i
            int_field = np.loadtxt(int_field_file, skiprows=1)
            print(f"Internal field data shape: {int_field.shape}")
            
            if int_field.shape[1] >= 10:
                # Columns: x y z |E|^2 Ex.r Ex.i Ey.r Ey.i Ez.r Ez.i
                # We want columns 4-9 for the complex field components
                Ex = int_field[:, 4] + 1j * int_field[:, 5]  # Ex.r + i*Ex.i
                Ey = int_field[:, 6] + 1j * int_field[:, 7]  # Ey.r + i*Ey.i  
                Ez = int_field[:, 8] + 1j * int_field[:, 9]  # Ez.r + i*Ez.i
                
                # Use the direct |E|^2 column from ADDA
                field_enhancement = int_field[:, 3]
                
                data['field_enhancement'] = field_enhancement
                data['E_field'] = np.column_stack([Ex, Ey, Ez])
                
                print(f"Field enhancement: min={np.min(field_enhancement):.3f}, "
                      f"max={np.max(field_enhancement):.3f}, mean={np.mean(field_enhancement):.3f}")
            else:
                print(f"Unexpected field file format: {int_field.shape}")
                return None
        else:
            print(f"Warning: {int_field_file} not found")
            return None
    except Exception as e:
        print(f"Error reading internal fields: {e}")
        return None
    
    # Read dipole polarizations
    try:
        if os.path.exists(pol_file):
            print(f"Reading polarizations from {pol_file}")
            
            # Skip header: x y z |P|^2 Px.r Px.i Py.r Py.i Pz.r Pz.i
            polarization = np.loadtxt(pol_file, skiprows=1)
            print(f"Polarization data shape: {polarization.shape}")
            
            if polarization.shape[1] >= 10:
                # Columns: x y z |P|^2 Px.r Px.i Py.r Py.i Pz.r Pz.i
                px = polarization[:, 4] + 1j * polarization[:, 5]  # Px.r + i*Px.i
                py = polarization[:, 6] + 1j * polarization[:, 7]  # Py.r + i*Py.i
                pz = polarization[:, 8] + 1j * polarization[:, 9]  # Pz.r + i*Pz.i
                
                data['polarization'] = np.column_stack([px, py, pz])
                
                # Calculate absorption and scattering contributions
                if 'E_field' in data:
                    # Absorption ∝ Im(p* · E)
                    absorption_contrib = np.imag(
                        np.conj(px) * data['E_field'][:, 0] +
                        np.conj(py) * data['E_field'][:, 1] +
                        np.conj(pz) * data['E_field'][:, 2]
                    )
                    data['absorption_contrib'] = absorption_contrib
                    print(f"Absorption contribution: min={np.min(absorption_contrib):.3e}, "
                          f"max={np.max(absorption_contrib):.3e}")
                
                # Scattering ∝ |p|² (we can also use the direct |P|^2 column)
                scattering_contrib = polarization[:, 3]  # Use direct |P|^2 from ADDA
                data['scattering_contrib'] = scattering_contrib
                print(f"Scattering contribution: min={np.min(scattering_contrib):.3e}, "
                      f"max={np.max(scattering_contrib):.3e}")
            else:
                print(f"Unexpected polarization file format: {polarization.shape}")
                return None
        else:
            print(f"Warning: {pol_file} not found")
            return None
    except Exception as e:
        print(f"Error reading polarizations: {e}")
        return None
    
    return data

def parse_cross_sections_from_file(run_dir="field_analysis"):
    """
    Parse cross-sections from ADDA output files
    Fixed to handle the text format correctly
    """
    crosssec_y_file = os.path.join(run_dir, "CrossSec-Y")
    crosssec_x_file = os.path.join(run_dir, "CrossSec-X")
    
    qext = qabs = qsca = None
    
    for crosssec_file in [crosssec_y_file, crosssec_x_file]:
        if os.path.exists(crosssec_file):
            try:
                print(f"Reading cross-sections from {crosssec_file}")
                with open(crosssec_file, 'r') as f:
                    lines = f.readlines()
                
                # Parse the text format: "Qext = 13.63176662"
                for line in lines:
                    line = line.strip()
                    if line.startswith('Qext'):
                        qext = float(line.split('=')[1].strip())
                    elif line.startswith('Qabs'):
                        qabs = float(line.split('=')[1].strip())
                        # Calculate Qsca = Qext - Qabs
                        if qext is not None:
                            qsca = qext - qabs
                
                if qext is not None and qabs is not None:
                    print(f"Cross-sections: Qext={qext:.3f}, Qabs={qabs:.3f}, Qsca={qsca:.3f}")
                    break
                    
            except Exception as e:
                print(f"Error reading {crosssec_file}: {e}")
    
    return qext, qabs, qsca


# Updated main analysis function
def analyze_nanocube_enhancement(shape_file, wavelength=408, n_rel_real=0.5, n_rel_imag=2.5):
    """
    Complete analysis of field enhancement and scattering/absorption contributions
    """
    print(f"=== Analyzing {shape_file} at {wavelength} nm ===")
    
    # Run ADDA with field extraction
    adda_result = analyze_single_wavelength_with_fields(
        shape_file, wavelength, n_rel_real=n_rel_real, n_rel_imag=n_rel_imag
    )
    
    if adda_result is None:
        print("ADDA calculation failed")
        return None
    
    # Parse cross-sections from files
    qext, qabs, qsca = parse_cross_sections_from_file()
    if qext is not None:
        adda_result.update({'qext': qext, 'qabs': qabs, 'qsca': qsca})
        print(f"Final cross-sections: Qext={qext:.3f}, Qabs={qabs:.3f}, Qsca={qsca:.3f}")
    
    # Read field and polarization data
    data = read_dipole_fields_and_polarizations()
    if data is None:
        print("Failed to read field data")
        return None
    
    # Create visualizations
    results = {
        'adda_result': adda_result,
        'data': data
    }
    
    # Field enhancement plot
    if 'field_enhancement' in data:
        print("Creating field enhancement visualization...")
        fig_enhancement = visualize_enhancement_3d(
            data['positions'], 
            data['field_enhancement'],
            f"Field Enhancement at {wavelength} nm (Qext={qext:.2f})"
        )
        results['fig_enhancement'] = fig_enhancement
        fig_enhancement.show()
    
    # Scattering contribution plot
    if 'scattering_contrib' in data:
        print("Creating scattering contribution visualization...")
        fig_scattering = visualize_enhancement_3d(
            data['positions'],
            data['scattering_contrib'], 
            f"Scattering Contribution at {wavelength} nm (Qsca={qsca:.2f})"
        )
        results['fig_scattering'] = fig_scattering
        fig_scattering.show()
    
    # Absorption contribution plot (absolute value)
    if 'absorption_contrib' in data:
        print("Creating absorption contribution visualization...")  
        fig_absorption = visualize_enhancement_3d(
            data['positions'],
            np.abs(data['absorption_contrib']), 
            f"Absorption Contribution at {wavelength} nm (Qabs={qabs:.2f})"
        )
        results['fig_absorption'] = fig_absorption
        fig_absorption.show()
    
    return results

# Let's also create a function to check what ADDA options are available
def check_adda_help(adda_path, option=""):
    """
    Check ADDA help for available options
    """
    try:
        if option:
            cmd = [adda_path, "-h", option]
        else:
            cmd = [adda_path, "-h"]
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=10)
        print(f"ADDA help for '{option}':")
        print(result.stdout)
        if result.stderr:
            print("Errors:")
            print(result.stderr)
            
    except Exception as e:
        print(f"Error getting ADDA help: {e}")

# Alternative approach: try minimal field extraction first
def analyze_single_wavelength_minimal(shape_file, wavelength=408, adda_path=None, 
                                    n_rel_real=0.5, n_rel_imag=2.5, run_dir="field_analysis"):
    """
    Minimal approach - just get geometry and basic field info
    """
    if adda_path is None:
        num_dipoles = get_n_dipoles_from_file(shape_file)
        adda_path = get_optimal_adda_path(num_dipoles)
    
    print(f"Running minimal ADDA with field extraction for {wavelength} nm...")
    
    # Start with minimal options and add more if they work
    cmd = [
        adda_path,
        "-shape", "read", shape_file,
        "-lambda", str(wavelength),
        "-m", f"{n_rel_real:.6f}", f"{n_rel_imag:.6f}",
        "-dir", run_dir,
        "-save_geom"  # Just save geometry first
    ]
    
    try:
        print(f"Running minimal command: {' '.join(cmd)}")
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        
        if result.returncode != 0:
            print(f"ADDA failed with error: {result.stderr}")
            return None
            
        print("Basic ADDA completed successfully")
        
        # List all files created
        print(f"\nFiles created in {run_dir}:")
        for file in os.listdir(run_dir):
            file_path = os.path.join(run_dir, file)
            if os.path.isfile(file_path):
                size = os.path.getsize(file_path)
                print(f"  {file} ({size} bytes)")
        
        return result
        
    except Exception as e:
        print(f"Error running ADDA: {e}")
        return None

# Let's also try different field extraction options
def try_field_extraction_options(shape_file, wavelength=408, adda_path=None, 
                               n_rel_real=0.5, n_rel_imag=2.5, run_dir="field_test"):
    """
    Try different field extraction options to see what works
    """
    if adda_path is None:
        num_dipoles = get_n_dipoles_from_file(shape_file)
        adda_path = get_optimal_adda_path(num_dipoles)
    
    # Create clean directory
    if os.path.exists(run_dir):
        import shutil
        shutil.rmtree(run_dir)
    os.makedirs(run_dir, exist_ok=True)
    
    # Test different options
    options_to_try = [
        # Option 1: Just internal surface
        ["-int_surf", "som"],
        ["-int_surf", "img"], 
        
        # Option 2: Try store options individually
        ["-store_int_field"],
        ["-store_dip_pol"],
        
        # Option 3: Try different field options
        ["-store_beam"],
        ["-store_scat_grid"],
    ]
    
    base_cmd = [
        adda_path,
        "-shape", "read", shape_file,
        "-lambda", str(wavelength),
        "-m", f"{n_rel_real:.6f}", f"{n_rel_imag:.6f}",
        "-dir", run_dir,
        "-save_geom"
    ]
    
    for i, options in enumerate(options_to_try):
        print(f"\n--- Testing option set {i+1}: {' '.join(options)} ---")
        
        cmd = base_cmd + options
        
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=60)
            
            if result.returncode == 0:
                print(f"✓ SUCCESS with options: {' '.join(options)}")
                print("Files created:")
                for file in os.listdir(run_dir):
                    print(f"  {file}")
            else:
                print(f"✗ FAILED with options: {' '.join(options)}")
                print(f"Error: {result.stderr}")
                
        except Exception as e:
            print(f"✗ EXCEPTION with options: {' '.join(options)}")
            print(f"Error: {e}")
        
        # Clean up for next test
        for file in os.listdir(run_dir):
            if file != "geometry.dat":  # Keep geometry file
                try:
                    os.remove(os.path.join(run_dir, file))
                except:
                    pass

def analyze_nanocube_enhancement_simple(shape_file, wavelength=408, run_dir="field_analysis"):
    """
    Simplified version that works with your existing functions
    """
    print(f"Analyzing {shape_file} at {wavelength} nm...")
    
    # Create run directory
    os.makedirs(run_dir, exist_ok=True)
    
    # Get ADDA path using your existing function
    num_dipoles = get_n_dipoles_from_file(shape_file)
    adda_path = get_optimal_adda_path(num_dipoles)
    
    # For silver, you'll need to get the proper refractive index
    # For now, using approximate values - you should use your material database
    n_rel_real = 0.5  # Approximate for Ag at 408 nm in water (n=1.33)
    n_rel_imag = 2.5  # Approximate imaginary part
    
    # Run ADDA with field extraction
    result = analyze_single_wavelength_with_fields(
        shape_file, wavelength, adda_path, n_rel_real, n_rel_imag, run_dir
    )
    
    if result is None:
        return None
    
    # Read the generated field data
    data = read_dipole_fields_and_polarizations(run_dir)
    if data is None:
        print("Could not read field data")
        return None
    
    # Create visualizations
    results = {'data': data}
    
    if 'field_enhancement' in data:
        fig_enhancement = visualize_enhancement_3d(
            data['positions'], 
            data['field_enhancement'],
            f"Field Enhancement at {wavelength} nm"
        )
        results['fig_enhancement'] = fig_enhancement
        fig_enhancement.show()
    
    if 'scattering_contrib' in data:
        fig_scattering = visualize_enhancement_3d(
            data['positions'],
            data['scattering_contrib'], 
            f"Scattering Contribution at {wavelength} nm"
        )
        results['fig_scattering'] = fig_scattering
        fig_scattering.show()
    
    return results



    """
    Simplified version that works with your existing functions
    """
    print(f"Analyzing {shape_file} at {wavelength} nm...")
    
    # Get ADDA path using your existing function
    num_dipoles = get_n_dipoles_from_file(shape_file)
    adda_path = get_optimal_adda_path(num_dipoles)
    
    # Run ADDA with field extraction
    result = analyze_single_wavelength_with_fields(shape_file, wavelength, adda_path)
    
    if result is None:
        return None
    
    # Read the generated field data
    data = read_dipole_fields_and_polarizations()
    if data is None:
        print("Could not read field data")
        return None
    
    # Create visualizations
    if 'field_enhancement' in data:
        fig = visualize_enhancement_3d(
            data['positions'], 
            data['field_enhancement'],
            f"Field Enhancement at {wavelength} nm"
        )
        fig.show()
        return {'data': data, 'fig': fig}
    
    return {'data': data}

In [ ]:
results = analyze_nanocube_enhancement_complete(
    "ag_np_100_10.dat", 
    wavelength=408, 
    metal='Ag',
    size_scale=8,      # Larger base size
    size_multiplier=9, # More size variation
    opacity=1.0,       # Less transparent
    colorscale='hot'
)

In [ ]:
num_dipoles = get_n_dipoles_from_file("ag_np_100_10.dat")
adda_path = get_optimal_adda_path(num_dipoles)

print("=== ADDA Help for int_surf ===")
check_adda_help(adda_path, "int_surf")

print("\n=== ADDA Help for store options ===")
check_adda_help(adda_path, "store")

print("\n=== Testing different field extraction options ===")
try_field_extraction_options("ag_np_100_10.dat", wavelength=408)